In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [10]:
import re
import json
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType, ArrayType, DoubleType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.linalg import Vectors
from pyspark.mllib.linalg import Vectors as MLLibVectors
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lit
from pyspark.sql.types import ArrayType, StringType, DoubleType
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.linalg import Vectors
import re
import numpy as np

In [3]:
import json
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import HashingTF, IDF
from scipy.spatial.distance import cosine
from pyspark.ml.linalg import DenseVector

In [4]:
given_courses = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'],
                 [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'],
                 [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'],
                 [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'],
                 [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'],
                 [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]

In [5]:
# Загрузка данных
data_path = "/labs/slaba02/DO_record_per_line.json"
data = spark.read.json(data_path)

# Функция для очистки текста
def clean_text(text):
    regex = re.compile(u'[\w\d]{2,}', re.U)
    return regex.findall(text.lower())

# Регистрация UDF
clean_text_udf = udf(clean_text, ArrayType(StringType()))

In [6]:
# Применение очистки текста к описаниям курсов
data = data.withColumn("clean_desc", clean_text_udf(col("desc")))

In [15]:
data.show(5)

+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
|                 cat|                desc| id|lang|                name|      provider|          clean_desc|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|[this, course, in...|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|[this, online, co...|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|[this, course, is...|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|[we, live, in, di...|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|[this, self, pace...|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
only showi

In [7]:
# Преобразование слов в векторы с помощью HashingTF
hashingTF = HashingTF(inputCol="clean_desc", outputCol="raw_features", numFeatures=10000)
featurizedData = hashingTF.transform(data)

In [19]:
featurizedData.show(5)

+--------------------+--------------------+---+----+--------------------+--------------+--------------------+--------------------+
|                 cat|                desc| id|lang|                name|      provider|          clean_desc|        raw_features|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+--------------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|[this, course, in...|(10000,[36,63,138...|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|[this, online, co...|(10000,[32,222,36...|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|[this, course, is...|(10000,[30,118,12...|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|[we, live, in, di...|(10000,[493,572,7...|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Net

In [8]:
# Вычисление IDF на основе частоты слов
idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [21]:
rescaledData.show(5)

+--------------------+--------------------+---+----+--------------------+--------------+--------------------+--------------------+--------------------+
|                 cat|                desc| id|lang|                name|      provider|          clean_desc|        raw_features|            features|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+--------------------+--------------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|[this, course, in...|(10000,[36,63,138...|(10000,[36,63,138...|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|[this, online, co...|(10000,[32,222,36...|(10000,[32,222,36...|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|[this, course, is...|(10000,[30,118,12...|(10000,[30,118,12...|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|

In [14]:
# Функция для вычисления косинусного сходства
def cosine_similarity(v1, v2):
    v1_np = np.array(v1)
    v2_np = np.array(v2)
    return float(np.dot(v1_np, v2_np) / (np.linalg.norm(v1_np) * np.linalg.norm(v2_np)))

# Регистрация UDF для косинусного сходства
cosine_similarity_udf = udf(cosine_similarity, DoubleType())

In [17]:
# Создание словаря с рекомендациями
recommendations = {}

# Обработка каждого курса
for course_id, lang, _ in given_courses:
    # Выбор курсов того же языка
    same_lang_courses = rescaledData.filter(col("lang") == lang)
    
    # Получение вектора для данного курса
    target_course_vector = same_lang_courses.filter(col("id") == course_id).select("features").collect()[0][0]
    
    # Создание Broadcast переменной для вектора данного курса
    target_course_vector_broadcast = spark.sparkContext.broadcast(target_course_vector.toArray())
    
    # Вычисление косинусного сходства
    def calculate_similarity(features):
        return cosine_similarity(target_course_vector_broadcast.value, features.toArray())
    
    calculate_similarity_udf = udf(calculate_similarity, DoubleType())
    
    similarities = same_lang_courses.withColumn("similarity", calculate_similarity_udf(col("features")))
    
    # Отбор топ-10 курсов
    top_courses = similarities.filter(col("id") != course_id).orderBy(col("similarity").desc(), col("name")).limit(10)
    recommended_ids = [row.id for row in top_courses.select("id").collect()]
    
    # Сохранение рекомендаций
    recommendations[str(course_id)] = recommended_ids

In [19]:
import json
with open("lab02.json", "w") as f:
    json.dump(recommendations, f)